In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

### Imports

In [2]:
import os
import pandas as pd
import tqdm
import re
import torch
import json
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np
import tensorflow as tf
from preprocess import *
from vocabulary import *
from model import *
from model_functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\swaga\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns
from string import punctuation
import re
from gensim.models import KeyedVectors
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Pre Processing Dataset

In [4]:
train_df = pd.read_csv('../../../evaluation/snli/snli_/snli_1.0_train.csv')
val_df   = pd.read_csv('../../../evaluation/snli/snli_/snli_1.0_dev.csv')
# train_df = pd.read_csv('../datasets/snli/snli_/snli_1.0_train.csv')
# val_df   = pd.read_csv('../datasets/snli/snli_/snli_1.0_dev.csv')
train_df, val_df = preprocess_text(train_df, val_df)
print(len(train_df))
print(len(val_df))

549361
9842


In [5]:
len(train_df['sentence2'][8])

36

In [6]:
# rw = pd.read_csv('../datasets/Transformer_generated_summaries.csv', encoding='utf-8')
# rw = pd.read_csv('ins.csv', encoding='utf-8')
rw = pd.read_csv('test_sets/ins.csv', encoding='utf-8')
# rw = pd.read_csv('../datasets/test_data/Transformer_generated_summaries.csv', encoding='utf-8')
rw.drop(['text'], axis = 1, inplace=True)
rw.rename(columns = {'original_summary':'sentence1'}, inplace = True)
rw.rename(columns = {'Predicted_summary':'sentence2'}, inplace = True)
gold = [0] * 100
rw.insert(loc=0, column='gold_label', value=gold)
rw = rw[['gold_label', 'sentence1', 'sentence2']]
rw = rw.dropna()
rw.head()

,gold_label,sentence1,sentence2
0,0,saudi stock exchange appoints its first female...,saudi stock exchange appointed sarah al ceo i...
1,0,what makes fireworks explode,hollywood actor morgan freeman said would lik...
2,0,60 british firms to increase investment in india,india business council chairperson said 60 br...
3,0,railways increase senior citizen women quota,indian railways wednesday increased lower ber...
4,0,pilots arrested in scotland for drinking befor...,two pilots arrested saturday glasgow scotland...


In [7]:
len(rw['sentence2'][8])

117

In [8]:
train_df['sentence1'] = train_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
train_df['sentence2'] = train_df['sentence2'].astype(str).apply(lambda text: clean_text(text))
val_df['sentence1'] = val_df['sentence1'].astype(str).apply(lambda text: clean_text(text))
val_df['sentence2'] = val_df['sentence2'].astype(str).apply(lambda text: clean_text(text))
rw['sentence1'] = rw['sentence1'].astype(str).apply(lambda text: clean_text(text))
rw['sentence2'] = rw['sentence2'].astype(str).apply(lambda text: clean_text(text))

In [9]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]
print(train_df[(train_df['sentence1'].str.split().str.len() == 0) | (train_df['sentence2'].str.split().str.len() == 0)])
print(val_df[(val_df['sentence1'].str.split().str.len() == 0) | (val_df['sentence2'].str.split().str.len() == 0)])

Empty DataFrame
Columns: [gold_label, sentence1, sentence2]
Index: []
Empty DataFrame
Columns: [gold_label, sentence1, sentence2]
Index: []


In [10]:
train_df.head()

,gold_label,sentence1,sentence2
0,neutral,a person on a horse jumps over a broken down a...,a person is training his horse for a competition
1,contradiction,a person on a horse jumps over a broken down a...,a person is at a diner ordering an omelette
2,entailment,a person on a horse jumps over a broken down a...,a person is outdoors on a horse
3,neutral,children smiling and waving at camera,they are smiling at their parents
4,entailment,children smiling and waving at camera,there are children present


In [11]:
val_df.head()

,gold_label,sentence1,sentence2
0,neutral,two women are embracing while holding to go pa...,the sisters are hugging goodbye while holding ...
1,entailment,two women are embracing while holding to go pa...,two woman are holding packages
2,contradiction,two women are embracing while holding to go pa...,the men are fighting outside a deli
3,entailment,two young children in blue jerseys one with th...,two kids in numbered jerseys wash their hands
4,neutral,two young children in blue jerseys one with th...,two kids at a ballgame wash their hands


In [12]:
train_val_df = pd.concat([train_df, val_df])

In [13]:
sentence_pairs, _ = pair_generator(train_val_df)
rw_sentence_pairs, __ = pair_generator(rw)
train_sentence_pairs, train_sentence_labels = pair_generator(train_df)
val_sentence_pairs, val_sentence_labels = pair_generator(val_df)

labels = set(train_sentence_labels)
print(labels)

tag2idx = {word: i for i, word in enumerate(labels)}
print(tag2idx)

train_labels = [tag2idx[t] for t in train_sentence_labels]
val_labels = [tag2idx[t] for t in val_sentence_labels]

{'entailment', 'neutral', 'contradiction'}
{'entailment': 0, 'neutral': 1, 'contradiction': 2}


In [14]:
vocab = Vocabulary()

for data in [rw_sentence_pairs]:
  for sen in data:
    premise    = sen[0]
    hypothesis = sen[1]
    vocab.addSentence(premise)
    vocab.addSentence(hypothesis)

for data in [sentence_pairs]:
  for sentence_pair in data:
    premise    = sentence_pair[0]
    hypothesis = sentence_pair[1]
    vocab.addSentence(premise)
    vocab.addSentence(hypothesis)

print("Vocab size:", len(vocab.word2index))

Vocab size: 33588


In [15]:
# vocab = Vocabulary()
print("Vocab size:", len(vocab.word2index))

Vocab size: 33588


In [16]:
index2word = {}
for wrd, idx in vocab.word2index.items():
    # print(wrd, idx)
    index2word[idx] = wrd

## Model

In [17]:
EPOCHS = 15
BATCH_SIZE = 32
EMBEDDING_SIZE = 200
VOCAB_SIZE = len(vocab.word2index)
TARGET_SIZE = len(tag2idx)
HIDDEN_SIZE = 64
LEARNING_RATE = 0.005
STACKED_LAYERS = 2
EMBEDDING_PATH = '../../../../embeddings/google_news/GoogleNews-vectors-negative300.bin'
GLOVE_EMBEDDING = '../../../../embeddings/glove/glove.6B.200d.txt'
# EMBEDDING_PATH = '../embeddings/google_news/GoogleNews-vectors-negative300.bin'
# GLOVE_EMBEDDING = '../embeddings/glove/glove.6B.300d.txt'

initiate_model_vocab(vocab, tag2idx)

In [18]:
train_data = DataSetLoader(get_pair_indices(vocab, train_sentence_pairs), train_labels)
val_data   = DataSetLoader(get_pair_indices(vocab, val_sentence_pairs), val_labels)

In [19]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)
val_loader   = torch.utils.data.DataLoader(val_data, batch_size = BATCH_SIZE, collate_fn=lambda x:x)

print(len(train_loader), len(val_loader))

17168 308


In [20]:
embeddings_index = load_embeddings(GLOVE_EMBEDDING)

In [21]:
weights = 1 * np.random.randn(VOCAB_SIZE + 1, EMBEDDING_SIZE)
embedded_count = 0
for word, lang_word_index in vocab.word2index.items():
  if embeddings_index.get(word) is not None:
    weights[lang_word_index] = embeddings_index.get(word)
    embedded_count += 1

print("Embedded count:", embedded_count)
del embeddings_index
weights.shape

Embedded count: 27414


(33589, 200)

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [23]:
lstm_model = LSTM(VOCAB_SIZE, HIDDEN_SIZE, TARGET_SIZE, STACKED_LAYERS, weights, True)
lstm_model.to(device)
print(lstm_model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

LSTM(
  (embedding): Embedding(33589, 200)
  (lstm): LSTM(200, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (FC_concat1): Linear(in_features=256, out_features=128, bias=True)
  (FC_concat2): Linear(in_features=128, out_features=64, bias=True)
  (FC_concat3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=3, bias=True)
  (out): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=32, out_features=3, bias=True)
  )
)


In [24]:
train(lstm_model, train_loader, val_loader, criterion, optimizer)

  7%|█████▎                                                                          | 1/15 [13:04<3:03:00, 784.36s/it]

00:13:04.35
Epoch 1: train_loss: 0.7954 train_acc: 0.6569 | val_loss: 0.7022 val_acc: 0.7123


 13%|██████████▋                                                                     | 2/15 [26:29<2:52:38, 796.83s/it]

00:13:25.56
Epoch 2: train_loss: 0.7266 train_acc: 0.6969 | val_loss: 0.6649 val_acc: 0.7278


 20%|████████████████                                                                | 3/15 [40:03<2:40:56, 804.68s/it]

...

00:12:15.39
Epoch 13: train_loss: 0.6826 train_acc: 0.7182 | val_loss: 0.6429 val_acc: 0.7399


 93%|█████████████████████████████████████████████████████████████████████████▋     | 14/15 [3:05:25<13:02, 782.16s/it]

00:12:26.17
Epoch 14: train_loss: 0.6867 train_acc: 0.7169 | val_loss: 0.6357 val_acc: 0.7399


100%|███████████████████████████████████████████████████████████████████████████████| 15/15 [3:17:30<00:00, 790.04s/it]

00:12:04.81
Epoch 15: train_loss: 0.6803 train_acc: 0.7197 | val_loss: 0.6382 val_acc: 0.7412


In [25]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in lstm_model.state_dict():
    print(param_tensor, "\t", lstm_model.state_dict()[param_tensor].size())

Model's state_dict:
embedding.weight 	 torch.Size([33589, 200])
lstm.weight_ih_l0 	 torch.Size([256, 200])
lstm.weight_hh_l0 	 torch.Size([256, 64])
lstm.bias_ih_l0 	 torch.Size([256])
lstm.bias_hh_l0 	 torch.Size([256])
lstm.weight_ih_l0_reverse 	 torch.Size([256, 200])
lstm.weight_hh_l0_reverse 	 torch.Size([256, 64])
lstm.bias_ih_l0_reverse 	 torch.Size([256])
lstm.bias_hh_l0_reverse 	 torch.Size([256])
lstm.weight_ih_l1 	 torch.Size([256, 128])
lstm.weight_hh_l1 	 torch.Size([256, 64])
lstm.bias_ih_l1 	 torch.Size([256])
lstm.bias_hh_l1 	 torch.Size([256])
lstm.weight_ih_l1_reverse 	 torch.Size([256, 128])
lstm.weight_hh_l1_reverse 	 torch.Size([256, 64])
lstm.bias_ih_l1_reverse 	 torch.Size([256])
lstm.bias_hh_l1_reverse 	 torch.Size([256])
FC_concat1.weight 	 torch.Size([128, 256])
FC_concat1.bias 	 torch.Size([128])
FC_concat2.weight 	 torch.Size([64, 128])
FC_concat2.bias 	 torch.Size([64])
FC_concat3.weight 	 torch.Size([32, 64])
FC_concat3.bias 	 torch.Size([32])
output.weigh

In [26]:
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Optimizer's state_dict:
state 	 {0: {'step': 257520, 'exp_avg': tensor([[-2.8026e-45, -0.0000e+00, -1.4013e-45,  ..., -2.8026e-45,
         -1.4013e-45, -4.2039e-45],
        [-4.2039e-45,  4.2039e-45, -5.6052e-45,  ..., -5.6052e-45,
          5.6052e-45, -2.9888e-37],
        [ 4.3737e-05,  7.0158e-05, -2.9655e-06,  ..., -1.8895e-05,
          1.8283e-06, -1.1721e-05],
        ...,
        [-2.8026e-45,  1.4013e-45, -1.4013e-45,  ...,  1.4013e-45,
         -5.6052e-45,  1.4013e-45],
        [-2.8026e-45,  1.4013e-45, -4.2039e-45,  ...,  1.4013e-45,
         -1.4013e-45,  2.8026e-45],
        [ 0.0000e+00, -1.4013e-45,  5.6052e-45,  ..., -2.8026e-45,
         -4.2039e-45,  1.4013e-45]], device='cuda:0'), 'exp_avg_sq': tensor([[7.0065e-43, 7.0065e-43, 7.0065e-43,  ..., 7.0065e-43, 7.0065e-43,
         7.0065e-43],
        [2.0277e-13, 3.8496e-13, 5.1157e-16,  ..., 2.2916e-13, 3.7312e-14,
         2.8334e-12],
        [9.0722e-10, 1.7469e-09, 5.8166e-10,  ..., 3.7395e-10, 2.5311e-10,
   

In [27]:
torch.save(lstm_model, 'sm_1')

In [29]:
lstm_model = torch.load('sm_1')
lstm_model.eval()

LSTM(
  (embedding): Embedding(33589, 200)
  (lstm): LSTM(200, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (FC_concat1): Linear(in_features=256, out_features=128, bias=True)
  (FC_concat2): Linear(in_features=128, out_features=64, bias=True)
  (FC_concat3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=3, bias=True)
  (out): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=32, out_features=3, bias=True)
  )
)

In [30]:
print(lstm_model.output)

Linear(in_features=32, out_features=3, bias=True)


In [31]:
sentence_o = [''] * len(rw)
sentence_p = [''] * len(rw)
# sentence_o = [''] * 2
# sentence_p = [''] * 2

# sentence_o[0] = "rock music is very good for public"
# sentence_p[0] = "people dont like rock and roll"
sentence_o[0] = "people like to listen to rock music"
sentence_p[0] = "rock and roll people love"
sentence_o[1] = "a girl is dancing on the stage"
sentence_p[1] = "girl dance"

sentence_o[0] = clean_text(sentence_o[0])
sentence_p[0] = clean_text(sentence_p[0])
sentence_o[1] = clean_text(sentence_o[1])
sentence_p[1] = clean_text(sentence_p[1])

sen_p = []
for i in range(2):
  sen1 = sentence_o[i]
  sen2 = sentence_p[i]
  sen_p.append((sen1, sen2))

sen_p

[('people like to listen to rock music', 'rock and roll people love'),
 ('a girl is dancing on the stage', 'girl dance')]

In [32]:
# rw_sentence_pairs[0]

In [39]:
id_pairs = get_pair_indices(vocab, rw_sentence_pairs)
# id_pairs = get_pair_indices(vocab, sen_p)
id_pairs[0]

([1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 9, 10, 11, 12, 13, 14, 15, 16, 6, 7])

In [40]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in id_pairs]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in id_pairs]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

100


In [41]:
prediction

tensor([[-0.8696,  0.6217, -0.0253],
        [-1.9065,  1.6477, -0.1347],
        [-0.0615,  0.3212, -0.4463],
        [-0.6877,  0.3944,  0.0492],
        [-0.6813,  0.5628, -0.1376],
        [-1.2927,  0.5959,  0.3359],
        [ 0.3494,  0.7090, -1.1463],
        [-1.0383,  0.5665,  0.1767],
        [-0.9075,  0.4701,  0.1568],
        [-0.9460,  1.6476, -0.9519],
        [-1.0585,  1.6333, -0.8350],
        [-1.2694,  1.6645, -0.6846],
        [-0.3276, -0.0769,  0.2976],
        [-1.0597,  0.5145,  0.2316],
        [-0.9506,  0.2287,  0.4170],
        [-1.0315, -0.0863,  0.9526],
        [ 0.1711,  0.9141, -1.1975],
        [-0.5653,  0.3290,  0.0131],
        [-1.0102, -0.5361,  1.8214],
        [-1.3167,  0.6405,  0.3191],
        [-0.7040,  0.2054,  0.3038],
        [-0.6242,  0.8319, -0.4530],
        [-0.8098,  0.8204, -0.2944],
        [-1.6101,  1.0189,  0.2162],
        [-0.6674,  0.6089, -0.2219],
        [ 0.6761,  0.8817, -1.5859],
        [ 0.4457,  0.5524, -1.0853],
 

In [42]:
soft = torch.log_softmax(prediction, dim=1).argmax(dim=1)
soft.tolist()

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 1]

In [43]:
rw_sentence_pairs[0]

('saudi stock exchange appoints its first female head',
 'saudi stock exchange appointed sarah al ceo investment bank ncb capital first female')

In [44]:
prediction[0]

tensor([-0.8696,  0.6217, -0.0253], device='cuda:0', grad_fn=<SelectBackward>)

In [45]:
rw_sentence_pairs[1]
# 0: entailment
# 1: neutral
# 2: contradiction

('what makes fireworks explode',
 'hollywood actor morgan freeman said would like make movie india morgan')

In [46]:
maxi = torch.max(prediction, dim=0)
mini = torch.min(prediction, dim=0)
# print(maxi[0][0])
denom1 = float(maxi[0][0].to('cpu')) - float(mini[0][0].to('cpu'))
denom2 = float(maxi[0][1].to('cpu')) - float(mini[0][1].to('cpu'))
denom3 = float(maxi[0][2].to('cpu')) - float(mini[0][2].to('cpu'))
denom = ( denom1 + denom2 + denom3 ) / 3
print(maxi, mini)
print(denom)

torch.return_types.max(
values=tensor([0.8695, 1.8687, 2.4898], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([83, 45, 81], device='cuda:0')) torch.return_types.min(
values=tensor([-3.2048, -0.7166, -1.8231], device='cuda:0', grad_fn=<MinBackward0>),
indices=tensor([45, 81, 33], device='cuda:0'))
3.657499452431997


In [47]:
aa = []
for i in prediction:
    a = []
    for j in i:
        a.append(float(j.to('cpu'))/denom)
    aa.append(a)
print(aa)

[[-0.23776825183719902, 0.16997312833714423, -0.006914738727502434], [-0.5212526456440189, 0.450503266169341, -0.03681542751502372], [-0.01681006866418081, 0.08781571963079066, -0.12202273668337109], [-0.1880111832399699, 0.1078229851786897, 0.01344966177176972], [-0.18626213906693254, 0.15387304358147239, -0.03760914720258072], [-0.35344127176150425, 0.1629380354691826, 0.09184596386591791], [0.0955209920691452, 0.1938372183191216, -0.31340734776402424], [-0.2838898521209764, 0.15488923162960536, 0.04830902996244051], [-0.2481276465776757, 0.1285250218000388, 0.042869467709664356], [-0.258649224643815, 0.4504627203379254, -0.26025042646050073], [-0.28941500388604957, 0.4465719175845136, -0.22829095708179742], [-0.3470685687101507, 0.4550870962647071, -0.18717882034391436], [-0.08957249651966674, -0.02101188304124699, 0.08135706039462925], [-0.2897435098626078, 0.1406793681119041, 0.06332127071309247], [-0.2599103499291575, 0.06252003831320324, 0.11401962838691664], [-0.282015096314683

In [48]:
e_scores = []
for i in aa:
    e_scores.append(i[0] + i[1]*0.5 + i[2]*0.1)
print(e_scores)

[-0.15347316154137713, -0.29968255531085075, 0.01489551748287741, -0.13275472447344808, -0.11308653199645441, -0.26278765764032114, 0.1610988664523036, -0.20161433330992967, -0.17957818890668983, -0.05944290712090239, -0.08895814080197252, -0.13824290261218855, -0.09194273200082731, -0.21307169873534648, -0.21724836793386423, -0.2677632214977649, 0.1390094534963514, -0.109244229830873, -0.29969530981043163, -0.26371216778282225, -0.15608466264567256, -0.06933021833718361, -0.11729719662568942, -0.29500454986264035, -0.10529325693326791, 0.2620268081062872, 0.16770760556196024, 0.10273246337790395, -0.25643505965014823, -0.32376438357801474, -0.2684535236481361, -0.20903967491630018, -0.1401314876759332, 0.16777069767865066, -0.38915895918785565, -0.1798634629719839, -0.011948664218833418, -0.3819004552035514, 0.021586471902904032, 0.18690065442894896, -0.27379676184715485, -0.33009976914570327, -0.2210191917750195, -0.1641740970587568, -0.3649827217122683, -0.6010803964789118, -0.35131

In [49]:
E_labels = []
for i in e_scores:
    if i < 0:
        E_labels.append('NEGATIVE ENTAILMENT')
    elif i > 0.2:
        E_labels.append('PERFECT ENTAILMENT')
    else:
        E_labels.append('NEUTRAL ENTAILMENT')

In [50]:
df_final_E = pd.DataFrame(list(zip(rw_sentence_pairs, aa, e_scores, E_labels)), columns =['sentence pairs', 'entailment scores (raw: e, n, c)', 'entailment_metric [-1:1]', 'entailment_labels'])
# df_final_E.to_csv('entailment_scores_of_test_pairs.csv')
df_final_E.head()

,sentence pairs,"entailment scores (raw: e, n, c)",entailment_metric [-1:1],entailment_labels
0,(saudi stock exchange appoints its first femal...,"[-0.23776825183719902, 0.16997312833714423, -0...",-0.153473,NEGATIVE ENTAILMENT
1,"(what makes fireworks explode, hollywood actor...","[-0.5212526456440189, 0.450503266169341, -0.03...",-0.299683,NEGATIVE ENTAILMENT
2,(british firms to increase investment in india...,"[-0.01681006866418081, 0.08781571963079066, -0...",0.014896,NEUTRAL ENTAILMENT
3,"(railways increase senior citizen women quota,...","[-0.1880111832399699, 0.1078229851786897, 0.01...",-0.132755,NEGATIVE ENTAILMENT
4,(pilots arrested in scotland for drinking befo...,"[-0.18626213906693254, 0.15387304358147239, -0...",-0.113087,NEGATIVE ENTAILMENT


In [52]:
# testcases_df = pd.read_csv('ins.csv')
# testcases_df.head()


In [56]:
ctr = 0
for i in range(len(soft)):
    if soft[i] == 2:
        print(rw_sentence_pairs[i])
        ctr = ctr + 1
print(ctr)

('arsenal beat sunderland to go top of premier league', 'arsenal alexis sanchez substitute olivier giroud scored brace secure')
('over killed in blast at shrine in balochistan', 'victims reportedly attending sufi dance called shrine blast occurred in balochistan')
('mercedes self driving car to save passengers not', 'executive von hugo said know save least one person least save one save one car')
('raveena govinda dance to mein at zee cine awards', 'video shows performing performing kisi disco mein film bade')
('nda slams nitish govt over law and order issue', 'opposition also brought alleged rape minor girl rjd mla raised anti government slogans following')
('would choose ranbir ranveer for ram remake jackie', 'actor jackie shroff said would choose actors ranbir kapoor ran')
('poor affects rd humans report', 'according global nutrition report poor nutrition affects third human population report states five billion adults')
('uefa chief to resign after losing appeal', 'european footbal

In [53]:
new_scores = {'sentence1': [], 'sentence2': []}
for i in testcases_df['original_summary']:
    new_scores['sentence2'].append(i)
for i in testcases_df['Predicted_summary']:
    new_scores['sentence1'].append(i)
for i in testcases_df['Predicted_summary']:
    new_scores['sentence1'].append(i)


NameError: name 'testcases_df' is not defined

In [ ]:
new_scores['sentence1'][0]

In [ ]:
sentence_o = [''] * len(testcases_df)
sentence_p = [''] * len(testcases_df)
# sentence_o = [''] * 2
# sentence_p = [''] * 2

# sentence_o[0] = "rock music is very good for public"
# sentence_p[0] = "people like rock and roll"
# sentence_o[1] = "a girl is dancing on the stage"
# sentence_p[1] = "hello world"

# sentence_o[0] = clean_text(sentence_o[0])
# sentence_p[0] = clean_text(sentence_p[0])
# sentence_o[1] = clean_text(sentence_o[1])
# sentence_p[1] = clean_text(sentence_p[1])

new_sen_p = []
for i in range(len(testcases_df)):
  sen1 = new_scores['sentence1'][i]
  sen2 = new_scores['sentence2'][i]
  new_sen_p.append((sen1, sen2))

new_sen_p

In [ ]:
for data in [new_sen_p]:
  for sentence_pair in data:
    premise    = sentence_pair[0]
    hypothesis = sentence_pair[1]
    vocab.addSentence(premise)
    vocab.addSentence(hypothesis)

In [ ]:
new_id_pairs = get_pair_indices(vocab, new_sen_p)

In [ ]:
premise_seq    = [torch.tensor(seq[0]).long().to(device) for seq in new_id_pairs]
hypothesis_seq = [torch.tensor(seq[1]).long().to(device) for seq in new_id_pairs]

premise_len    = list(map(len, premise_seq))
hypothesis_len = list(map(len, hypothesis_seq))

batch = len(premise_seq)
temp = pad_sequence(premise_seq + hypothesis_seq, batch_first=True)
premise_seq    = temp[:batch, :]
hypothesis_seq = temp[batch:, :]

prediction = lstm_model([premise_seq, hypothesis_seq], premise_len, hypothesis_len)
# prediction = prediction[prediction!=prediction[0,3]]

prediction_list = prediction.to('cpu')
prediction_list = prediction_list.tolist()
print(len(prediction_list))

In [ ]:
# soft = torch.log_softmax(prediction, dim=1).argmax(dim=1)
# soft.tolist()

In [ ]:
# ctr = 0
# for i in range(len(soft)):
#     if soft[i] == 1:
#         print(rw_sentence_pairs[i])
#         ctr = ctr + 1
# print(ctr)